# F1 Safety Car Predictor

## Authors: Petar Stamenković, Aleksa Mitrovčan

## Import new table with weather track data.

In [13]:
import numpy as np
import pandas as pd
safety_car_predictor = pd.read_csv("safety_car_predictor.csv")
circuit_data = pd.read_csv("circuit_data.csv")
safety_car_predictor
circuit_data
#safety_car_predictor

,name,circuit_id,circuit_type,phy_diff
0,Australian Grand Prix,1,street,2
1,Malaysian Grand Prix,2,traditional,4
2,Bahrain Grand Prix,3,traditional,3
3,Spanish Grand Prix,4,traditional,2
4,Turkish Grand Prix,5,traditional,2
5,Monaco Grand Prix,6,street,3
6,Canadian Grand Prix,7,traditional,2
7,French Grand Prix,8,traditional,2
8,British Grand Prix,9,traditional,1
9,German Grand Prix,10,traditional,2


## An attempt to use a shrinked table as fact-table. Merge two tables

In [14]:
#merged_table = safety_car_predictor_shrinked.join(circuit_data, on='name')
safety_car_predictor = pd.merge(safety_car_predictor,circuit_data, on='name')
safety_car_predictor

,year,name,date,weather,num_dnfs,safety_car,num_of_sc,num_of_vsc,total,circuit_id,circuit_type,phy_diff
0,2002,Australian Grand Prix,3/3/2002,dry,12,True,2.0,NaN,NaN,1,street,2
1,2003,Australian Grand Prix,3/9/2003,dry,9,True,2.0,NaN,NaN,1,street,2
2,2004,Australian Grand Prix,3/7/2004,dry,5,False,0.0,NaN,NaN,1,street,2
3,2005,Australian Grand Prix,3/6/2005,dry,5,False,0.0,NaN,NaN,1,street,2
4,2006,Australian Grand Prix,4/2/2006,dry,10,True,4.0,NaN,NaN,1,street,2
...,...,...,...,...,...,...,...,...,...,...,...,...
427,2022,Miami Grand Prix,5/8/2022,dry,4,True,1.0,1.0,NaN,34,street,3
428,2023,Miami Grand Prix,5/7/2023,dry,0,False,0.0,0.0,NaN,34,street,3
429,2024,Miami Grand Prix,5/5/2024,dry,1,True,1.0,1.0,NaN,34,street,3
430,2023,Las Vegas Grand Prix,11/19/2023,dry,3,True,NaN,NaN,NaN,35,street,3


## Unique list of all circuits

In [15]:
safety_car_predictor.name.unique() # List out all circuits in order to create a new DataFrame for circuit ID

array(['Australian Grand Prix', 'Malaysian Grand Prix',
       'Brazilian Grand Prix', 'Emilia Romagna Grand Prix',
       'Spanish Grand Prix', 'Austrian Grand Prix', 'Monaco Grand Prix',
       'Canadian Grand Prix', 'European Grand Prix', 'British Grand Prix',
       'French Grand Prix', 'German Grand Prix', 'Hungarian Grand Prix',
       'Belgian Grand Prix', 'Italian Grand Prix',
       'Indianapolis Grand Prix', 'Japanese Grand Prix',
       'Bahrain Grand Prix', 'Chinese Grand Prix', 'Turkish Grand Prix',
       'Singapore Grand Prix', 'Abu Dhabi Grand Prix',
       'Korean Grand Prix', 'United States Grand Prix',
       'Russian Grand Prix', 'Mexican Grand Prix',
       'Azerbaijan Grand Prix', 'Tuscan Grand Prix', 'Eifel Grand Prix',
       'Qatar Grand Prix', 'Dutch Grand Prix', 'Saudi Arabian Grand Prix',
       'Miami Grand Prix', 'Las Vegas Grand Prix'], dtype=object)

## Creating a new DataFrame for mapping circuitID and safety car predictor table

In [16]:

circuit_mapping = {
    'Australian Grand Prix': 1,
    'Malaysian Grand Prix': 2,
    'Chinese Grand Prix': 3,
    'Bahrain Grand Prix': 4,
    'Spanish Grand Prix': 5,
    'Monaco Grand Prix': 6,
    'Canadian Grand Prix': 7,
    'British Grand Prix': 8,
    'German Grand Prix': 9,
    'Hungarian Grand Prix': 10,
    'Belgian Grand Prix': 11,
    'Italian Grand Prix': 12,
    'Singapore Grand Prix': 13,
    'Korean Grand Prix': 14,
    'Japanese Grand Prix': 15,
    'Indian Grand Prix': 16,
    'Abu Dhabi Grand Prix': 17,
    'United States Grand Prix': 18,
    'Brazilian Grand Prix': 19,
    'Austrian Grand Prix': 20,
    'Russian Grand Prix': 21,
    'Mexican Grand Prix': 22,
    'Azerbaijan Grand Prix': 23,
    'French Grand Prix': 24,
    'Tuscan Grand Prix': 25,
    'Eifel Grand Prix': 26,
    'Portuguese Grand Prix': 27,
    'Emilia Romagna Grand Prix': 28,
    'Turkish Grand Prix': 29,
    'Qatar Grand Prix': 30,
    'Dutch Grand Prix': 31,
    'Mexico City Grand Prix': 32,
    'Sao Paulo Grand Prix': 33,
    'Saudi Arabian Grand Prix': 34,
    'Miami Grand Prix': 35,
    'Las Vegas Grand Prix': 36,
    'Indianapolis Grand Prix' : 37,
}

#safety_car_predictor.insert(0, 'circuit_id', safety_car_predictor['name'].map(circuit_mapping))
safety_car_predictor

,year,name,date,weather,num_dnfs,safety_car,num_of_sc,num_of_vsc,total,circuit_id,circuit_type,phy_diff
0,2002,Australian Grand Prix,3/3/2002,dry,12,True,2.0,NaN,NaN,1,street,2
1,2003,Australian Grand Prix,3/9/2003,dry,9,True,2.0,NaN,NaN,1,street,2
2,2004,Australian Grand Prix,3/7/2004,dry,5,False,0.0,NaN,NaN,1,street,2
3,2005,Australian Grand Prix,3/6/2005,dry,5,False,0.0,NaN,NaN,1,street,2
4,2006,Australian Grand Prix,4/2/2006,dry,10,True,4.0,NaN,NaN,1,street,2
...,...,...,...,...,...,...,...,...,...,...,...,...
427,2022,Miami Grand Prix,5/8/2022,dry,4,True,1.0,1.0,NaN,34,street,3
428,2023,Miami Grand Prix,5/7/2023,dry,0,False,0.0,0.0,NaN,34,street,3
429,2024,Miami Grand Prix,5/5/2024,dry,1,True,1.0,1.0,NaN,34,street,3
430,2023,Las Vegas Grand Prix,11/19/2023,dry,3,True,NaN,NaN,NaN,35,street,3


## Calculate average number of dnfs for each circuit and create a separate column

In [17]:
safety_car_predictor['avg_dnfs_per_circuit'] = safety_car_predictor.groupby('name')['num_dnfs'].transform('mean')
safety_car_predictor

,year,name,date,weather,num_dnfs,safety_car,num_of_sc,num_of_vsc,total,circuit_id,circuit_type,phy_diff,avg_dnfs_per_circuit
0,2002,Australian Grand Prix,3/3/2002,dry,12,True,2.0,NaN,NaN,1,street,2,6.761905
1,2003,Australian Grand Prix,3/9/2003,dry,9,True,2.0,NaN,NaN,1,street,2,6.761905
2,2004,Australian Grand Prix,3/7/2004,dry,5,False,0.0,NaN,NaN,1,street,2,6.761905
3,2005,Australian Grand Prix,3/6/2005,dry,5,False,0.0,NaN,NaN,1,street,2,6.761905
4,2006,Australian Grand Prix,4/2/2006,dry,10,True,4.0,NaN,NaN,1,street,2,6.761905
...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,2022,Miami Grand Prix,5/8/2022,dry,4,True,1.0,1.0,NaN,34,street,3,1.666667
428,2023,Miami Grand Prix,5/7/2023,dry,0,False,0.0,0.0,NaN,34,street,3,1.666667
429,2024,Miami Grand Prix,5/5/2024,dry,1,True,1.0,1.0,NaN,34,street,3,1.666667
430,2023,Las Vegas Grand Prix,11/19/2023,dry,3,True,NaN,NaN,NaN,35,street,3,2.500000


## Calculate average number of safety cars and virtual safety cars per race

In [18]:
safety_car_predictor['avg_sc_per_circuit'] = safety_car_predictor.groupby('name')['num_of_sc'].transform('mean')
safety_car_predictor['avg_vsc_per_circuit'] = safety_car_predictor.groupby('name')['num_of_vsc'].transform('mean')
#rearange columns
columns = ['year', 'name', 'circuit_id', 'circuit_type', 'phy_diff', 'date', 'weather', 'num_dnfs', 'avg_dnfs_per_circuit', 'num_of_sc', 'num_of_vsc', 'total', 'avg_sc_per_circuit', 'avg_vsc_per_circuit', 
           'safety_car']
safety_car_predictor = safety_car_predictor[columns]
safety_car_predictor


,year,name,circuit_id,circuit_type,phy_diff,date,weather,num_dnfs,avg_dnfs_per_circuit,num_of_sc,num_of_vsc,total,avg_sc_per_circuit,avg_vsc_per_circuit,safety_car
0,2002,Australian Grand Prix,1,street,2,3/3/2002,dry,12,6.761905,2.0,NaN,NaN,1.095238,0.625000,True
1,2003,Australian Grand Prix,1,street,2,3/9/2003,dry,9,6.761905,2.0,NaN,NaN,1.095238,0.625000,True
2,2004,Australian Grand Prix,1,street,2,3/7/2004,dry,5,6.761905,0.0,NaN,NaN,1.095238,0.625000,False
3,2005,Australian Grand Prix,1,street,2,3/6/2005,dry,5,6.761905,0.0,NaN,NaN,1.095238,0.625000,False
4,2006,Australian Grand Prix,1,street,2,4/2/2006,dry,10,6.761905,4.0,NaN,NaN,1.095238,0.625000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,2022,Miami Grand Prix,34,street,3,5/8/2022,dry,4,1.666667,1.0,1.0,NaN,0.666667,0.666667,True
428,2023,Miami Grand Prix,34,street,3,5/7/2023,dry,0,1.666667,0.0,0.0,NaN,0.666667,0.666667,False
429,2024,Miami Grand Prix,34,street,3,5/5/2024,dry,1,1.666667,1.0,1.0,NaN,0.666667,0.666667,True
430,2023,Las Vegas Grand Prix,35,street,3,11/19/2023,dry,3,2.500000,NaN,NaN,NaN,NaN,NaN,True


## Data modification for model training 

In [19]:
# safety_car_predictor_circuit = safety_car_predictor.loc[safety_car_predictor.name == query_name].copy()

# Replace string values in the 'weather' column
safety_car_predictor['weather'] = safety_car_predictor['weather'].replace({
    "dry": 0,
    "light_rain": 1,
    "heavy_rain": 2
})

# Replace string values in the 'circuit type' column
safety_car_predictor['circuit_type'] = safety_car_predictor['circuit_type'].replace({
    "traditional" : 0,
    "street" : 1
})

safety_car_predictor


,year,name,circuit_id,circuit_type,phy_diff,date,weather,num_dnfs,avg_dnfs_per_circuit,num_of_sc,num_of_vsc,total,avg_sc_per_circuit,avg_vsc_per_circuit,safety_car
0,2002,Australian Grand Prix,1,1,2,3/3/2002,0,12,6.761905,2.0,NaN,NaN,1.095238,0.625000,True
1,2003,Australian Grand Prix,1,1,2,3/9/2003,0,9,6.761905,2.0,NaN,NaN,1.095238,0.625000,True
2,2004,Australian Grand Prix,1,1,2,3/7/2004,0,5,6.761905,0.0,NaN,NaN,1.095238,0.625000,False
3,2005,Australian Grand Prix,1,1,2,3/6/2005,0,5,6.761905,0.0,NaN,NaN,1.095238,0.625000,False
4,2006,Australian Grand Prix,1,1,2,4/2/2006,0,10,6.761905,4.0,NaN,NaN,1.095238,0.625000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,2022,Miami Grand Prix,34,1,3,5/8/2022,0,4,1.666667,1.0,1.0,NaN,0.666667,0.666667,True
428,2023,Miami Grand Prix,34,1,3,5/7/2023,0,0,1.666667,0.0,0.0,NaN,0.666667,0.666667,False
429,2024,Miami Grand Prix,34,1,3,5/5/2024,0,1,1.666667,1.0,1.0,NaN,0.666667,0.666667,True
430,2023,Las Vegas Grand Prix,35,1,3,11/19/2023,0,3,2.500000,NaN,NaN,NaN,NaN,NaN,True


## Random forrest

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
#columns = ['year', 'name', 'circuit_id', 'date', 'weather', 'num_dnfs', 'avg_dnfs_per_circuit', 'num_of_sc', 'num_of_vsc', 'total', 'avg_sc_per_circuit', 'avg_vsc_per_circuit', 
#           'safety_car']
circuit_names = ['Australian Grand Prix', 'Malaysian Grand Prix',
       'Brazilian Grand Prix', 'Emilia Romagna Grand Prix',
       'Spanish Grand Prix', 'Austrian Grand Prix', 'Monaco Grand Prix',
       'Canadian Grand Prix', 'European Grand Prix', 'British Grand Prix',
       'French Grand Prix', 'German Grand Prix', 'Hungarian Grand Prix',
       'Belgian Grand Prix', 'Italian Grand Prix',
       'Indianapolis Grand Prix', 'Japanese Grand Prix',
       'Bahrain Grand Prix', 'Chinese Grand Prix', 'Turkish Grand Prix',
       'Singapore Grand Prix', 'Abu Dhabi Grand Prix',
       'Korean Grand Prix', 'United States Grand Prix',
       'Russian Grand Prix', 'Mexican Grand Prix',
       'Azerbaijan Grand Prix', 'Tuscan Grand Prix', 'Eifel Grand Prix',
       'Qatar Grand Prix', 'Dutch Grand Prix', 'Saudi Arabian Grand Prix',
       'Miami Grand Prix', 'Las Vegas Grand Prix']
features = ['circuit_id', 'weather', 'circuit_type', 'phy_diff','avg_dnfs_per_circuit', 'avg_sc_per_circuit', 'avg_vsc_per_circuit']
X = safety_car_predictor[features]
y = safety_car_predictor.safety_car

train_X, val_x, train_y, val_y = train_test_split(X,y,random_state = 0)

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)

# Prediction sample has to be a 2-dimensional array [x,y,z]!
# x is a weather parameter and it maps to : 0 - dry , 1 - light rain , 2 - heavy rain
# y is a type of circuit parameter and it maps to : 0 - traditional , 1 - street
# z is a physical difficulty of circuit parameter and it has 4 levels

# Enter circuit name (ex. Australian Grand Prix) for the prediction
# Manually edit weather forecast (0 - dry, 1 - light rain, 2 - heavy rain)


while True:
    circuit_name = input("Enter circuit name (ex. Australian Grand Prix) for the prediction : ").strip()
    if circuit_name in circuit_names:
        print(f"Valid circuit: {circuit_name}")
        break
    else:
        print("Invalid circuit name! Please enter a valid Grand Prix name.")
        
while True:
    valid_weather=[0,1,2]
    weather = int(input("Enter weather forecast (0 - dry, 1 - light rain, 2 - heavy rain)"))
    if weather in valid_weather:
        print(f"Valid weather input: {weather}")
        break
    else:
        print("Invalid weather! Please enter a valid value.")
prediction_sample = safety_car_predictor[safety_car_predictor['name'] == circuit_name][['circuit_id', 'circuit_type', 'phy_diff','avg_dnfs_per_circuit','avg_sc_per_circuit', 'avg_vsc_per_circuit']]
prediction_sample = prediction_sample.iloc[0].tolist()
prediction_sample.insert(1, weather)
print(prediction_sample)

#prediction_sample = [[1,1,1,2,6.761905, 1.095238, 0.625000]] # For instance this predicts a safety car chance for Australian Grand Prix with light rain!
safety_car_deployment_prediction = forest_model.predict([prediction_sample]) * 100
safety_car_deployment_prediction_rounded = np.round(safety_car_deployment_prediction,2)


print('\033[92m' + '\033[1m' + f"Prediction : {safety_car_deployment_prediction_rounded}%")



Enter circuit name (ex. Australian Grand Prix) for the prediction : Australian Grand Prix
Valid circuit: Australian Grand Prix
Enter weather forecast (0 - dry, 1 - light rain, 2 - heavy rain)1
Valid weather input: 1
[1.0, 1, 1.0, 2.0, 6.761904761904762, 1.0952380952380953, 0.625]
Prediction : [75.62]%


/usr/lib/python3/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [21]:
print(safety_car_predictor.groupby('weather')['safety_car'].mean()) # This shows what is the effect of a weather on a target variable


weather
0    0.514745
1    0.567568
2    0.863636
Name: safety_car, dtype: float64


In [22]:
for column in range(len(features)):
    print(features[column], forest_model.feature_importances_[column]) # This command shows how important each column is for our model


circuit_id 0.31189165548956543
weather 0.22999141985575128
circuit_type 0.08597381476670504
phy_diff 0.11610458114964406
avg_dnfs_per_circuit 0.20879518337151473
avg_sc_per_circuit 0.018705470116363823
avg_vsc_per_circuit 0.02853787525045564
